In [ ]:
import tensorflow
import keras
import pandas
import sklearn
# If version issues: print(tensorflow.__version__)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import glob, os
from keras.preprocessing.sequence import pad_sequences
import random, copy
import keras
import math
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from keras.preprocessing.text import hashing_trick

path = "/PATH/TO/Datasets/" # Replace with appropriate path
dfTotal = pd.read_csv(path+"Combined_News.csv", usecols=['title', 'label'])

def binaryLabel(label):
  if label == "TRUE":
    return 1
  return 0

dfTotal['label'] = dfTotal['label'].apply(lambda label: binaryLabel(str(label)))

X = dfTotal['title'].values
y = dfTotal['label'].values

os.chdir("/PATH/TO/Models/") # Replace with appropriate path

classifier = keras.models.load_model('CoAID_using_HashingTrick.h5')
X_temp =  [hashing_trick(str(elem), 10000, hash_function='md5') for elem in X]
X_temp = pad_sequences(X_temp, padding='post', maxlen=500)
modelPrediction = classifier.predict(X_temp)
modelPrediction = [math.floor(0.5+pred) for pred in modelPrediction]
print("Results for model trained on CoAID (complete) and evaluated on Poynter + Various using threshhold of >=0.5 is positive")
print(classification_report(y, modelPrediction))

# Testing on CoAID as well 
CoAID_dfFalse = pd.read_csv(path+"CoAID/NewsFakeCOVID-19.csv", usecols=['title'])
CoAID_dfFalse['label']=0
CoAID_dfFalseJuly = pd.read_csv(path+"CoAID/NewsFakeCOVID-19-JULY.csv", usecols=['title'])
CoAID_dfFalseJuly['label']=0
CoAID_dfTrue = pd.read_csv(path+"CoAID/NewsRealCOVID-19.csv", usecols=['title'], nrows=len(CoAID_dfFalse.values))
CoAID_dfTrue['label']=1
CoAID_dfTrueJuly = pd.read_csv(path+"CoAID/NewsRealCOVID-19-JULY.csv", usecols=['title'], nrows=len(CoAID_dfFalseJuly.values))
CoAID_dfTrueJuly['label']=1
CoAID_dfTotal = pd.concat([CoAID_dfTrue, CoAID_dfFalse, CoAID_dfTrueJuly, CoAID_dfFalseJuly])

CoAID_X = CoAID_dfTotal['title'].values
CoAID_y = CoAID_dfTotal['label'].values

classifier = keras.models.load_model('CoAID_using_HashingTrick.h5')
CoAID_X_temp =  [hashing_trick(str(elem), 10000, hash_function='md5') for elem in CoAID_X]
CoAID_X_temp = pad_sequences(CoAID_X_temp, padding='post', maxlen=500)
modelPrediction = classifier.predict(CoAID_X_temp)
modelPrediction = [math.floor(0.5+pred) for pred in modelPrediction]
print("Results for model trained on CoAID and tested on CoAID")
print(classification_report(CoAID_y, modelPrediction))

